In [ ]:
import sys
import os
import datetime
import pandas as pd
import seaborn as sns
from keras import backend as K
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000   
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from src.models.params import get_params
from src.models.model_utils import evaluate_test_set, write_csv_files
from src.models.Unet import Unet
from src.visualization.make_image_files import visualize_test_data, visualize_landsat8_tile
from src.visualization.visualization_utils import get_predicted_thumbnails 
from src.utils import get_model_name

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

/home/jhj/anaconda3/envs/CloudDetection/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


# Define and run the training loops

In [ ]:
activation_functions = ['swish', 'relu']
loss_functions = ['binary_crossentropy', 'jaccard_coef_loss']
optimizers = ['Adam', 'Nadam']
dropout_vals = [0, 0.5]
epochs = 300
num_gpus = 1

# Train the models
for activation_func in activation_functions:
    for loss_function in loss_functions:
        for optimizer in optimizers:
            for dropout in dropout_vals:
                K.clear_session()
                params = get_params('U-net', 'Landsat8')
                params.modelID = datetime.datetime.now().strftime("%y%m%d%H%M%S")
                params.loss_func = loss_function
                params.optimizer = optimizer
                params.activation_func = activation_func
                params.dropout = dropout
                params.epochs = epochs
                params.brightness_augmentation = False
                params.batch_size = 12
                params.learning_rate = 2e-2

                print('###########################################')
                print('# activation_func: ' + activation_func)
                print('# loss_function:   ' + loss_function)
                print('# optimizer:       ' + optimizer)
                print('###########################################')
                model = Unet()
                model.train(num_gpus, params)
                evaluate_test_set(model, num_gpus, params)
            

###########################################
# activation_func: swish
# loss_function:   binary_crossentropy
# optimizer:       Adam
###########################################
------------------------------------------
Always training from scratch at the moment
------------------------------------------
Start training:
Epoch 1/300
40/40 [==============================] - 25s 615ms/step - loss: 0.4690 - binary_crossentropy: 0.4690 - jaccard_coef_loss: 5.2791 - jaccard_coef: 0.5014 - jaccard_coef_thresholded: 0.6050 - acc: 0.7848 - val_loss: 1.2790 - val_binary_crossentropy: 1.2790 - val_jaccard_coef_loss: 8.2553 - val_jaccard_coef: 0.3612 - val_jaccard_coef_thresholded: 0.4507 - val_acc: 0.6502
Epoch 2/300
40/40 [==============================] - 21s 514ms/step - loss: 0.4176 - binary_crossentropy: 0.4176 - jaccard_coef_loss: 4.8583 - jaccard_coef: 0.5053 - jaccard_coef_thresholded: 0.6167 - acc: 0.8108 - val_loss: 1.7095 - val_binary_crossentropy: 1.7095 - val_jaccard_coef_loss: 9.8103

In [ ]:
classes = [['thin', 'cloud']]
epochs = 50
num_gpus = 1

# Train the models
for cls in classes:
        print('###########################################')
        print('# class:   ' + str(cls))
        print('###########################################')
        params = get_params('U-net', 'Landsat8')
        params.batch_size = 12
        params.modelID = datetime.datetime.now().strftime("%y%m%d%H%M%S")
        params.loss_func = 'binary_crossentropy'
        params.optimizer = 'Nadam'
        params.activation_func = 'swish'
        params.epochs = epochs
        params.cls = cls
        
        model = Unet()
        model.train(num_gpus, params)

        # Test different threshold values for the trained model (remember timeID is the same for all thresholds)
        for threshold in thresholds:            
            params.threshold = threshold
            avg_jaccard, product_names, product_jaccard = evaluate_test_set(model, num_gpus, params)
            write_csv_files(avg_jaccard, product_jaccard, product_names, params) 

# Investigate the trained models

In [ ]:
params = get_params('U-net', 'Landsat8')
df = pd.read_csv(params.project_path + 'reports/Unet/param_optimization.csv' )
#df.sort_values('entire_testset', ascending=False).head(40)
#df.loc[df['modelID'] == 180201172308]
df.loc[df['cls'] == 'shadow']


# Plot predictions from the desired model

In [ ]:
modelID = '180202154744'
num_gpus = 2

K.clear_session()
model = Unet()
params = get_params('U-net', 'Landsat8')
params.add_hparam('modelID', modelID)
params.activation_func = 'swish'
params.cls = ['shadow']
visualize_test_data(model, num_gpus, params)

In [ ]:
# Overlay pictures 
thresholded = True
transparency = 200
thumbnail_res = 512, 512  # Resolution to be showed
area = (1000, 1000, 7000, 7000)  # Area to be cropped in (min_width, min_height, max_width, max_height)
params.threshold = 0.1
model_name = get_model_name(params)

# Plot predictions
files = sorted(os.listdir(params.project_path + 'data/output/'))  # os.listdir loads in arbitrary order, hence use sorted()
files = [f for f in files if ('thresholded_Unet_Landsat8_' + modelID) in f]  # Filter out one ID for each tile
    
for i, f in enumerate(files, start=1):
    rgb, pred_unet, pred_true = get_predicted_thumbnails(f, thresholded, area, transparency, thumbnail_res, params)

    # Plot
    plt.figure(figsize=(35, 35))
    
    plt.subplot(1, 3, 1)
    plt.title('RGB for tile: ' + f[0:21])
    plt.imshow(rgb)
    
    plt.subplot(1, 3, 2)
    plt.title('Unet for tile: ' + f[0:21])
    plt.imshow(pred_unet)
    
    plt.subplot(1, 3, 3)
    plt.title('True for tile: ' + f[0:21])
    plt.imshow(pred_true)

    plt.show()